In [1]:
from abc import ABCMeta, abstractstaticmethod
import json
import requests
from pyproj import Proj, transform

In [2]:
class Coordinates(metaclass=ABCMeta):
    """ An abstract class represents the coordinates system """
    wgs = Proj(init='epsg:4326')
    mercator = Proj(init='epsg:3857')
    
    def __init__(self, long, lat):
        self.long = long
        self.lat = lat
    
    @staticmethod
    @abstractstaticmethod
    def from_wgs_coordinates(long, lat):
        pass
    
    @staticmethod
    @abstractstaticmethod
    def from_mercator_coordinates(long, lat):
        pass
    
    def __repr__(self):
        return f'Longitude is : {self.long} , Latitude is {self.lat}'

class WGSCoordinates(Coordinates):
    """ WGS coordinates system """
    def __init__(self, long, lat):
        super().__init__(long, lat)
        
    @staticmethod
    def from_wgs_coordinates(long, lat):
        return WGSCoordinates(long, lat)
    
    @staticmethod
    def from_mercator_coordinates(long, lat):
        wgs_point = transform(Coordinates.mercator, Coordinates.wgs, long, lat)
        return WGSCoordinates(wgs_point[0], wgs_point[1])

class MercatorCoordinates(Coordinates):
    """ Mercator coordinates system """
    def __init__(self, long, lat):
        super().__init__(long, lat)
    
    @staticmethod
    def from_wgs_coordinates(long, lat):
        mercator_point = transform(Coordinates.wgs, Coordinates.mercator, long, lat)
        return MercatorCoordinates(mercator_point[0], mercator_point[1])
    
    @staticmethod
    def from_mercator_coordinates(long, lat):
        return MercatorCoordinates(long, lat)

In [3]:
class BoundingBox(metaclass=ABCMeta):
    """ An abstract class represents the bounding box """
    def __init__(self, upper_left, bottom_right):
        self.upper_left = upper_left
        self.bottom_right = bottom_right
    
    # Sequence is [upper_left_long, upper_left_lat, bottom_right_long, bottom_right_lat]
    @staticmethod
    @abstractstaticmethod
    def bbox_from_wgs_sequence(seq):
        pass
    
    @staticmethod
    @abstractstaticmethod
    def bbox_from_mercator_sequence(seq):
        pass
    
    @staticmethod
    def wgs_bbox_from_openstreetmap_server(country):
        r = requests.get(r"http://nominatim.openstreetmap.org/search?q=%s&format=json"%country)
        bbox = json.loads(r.text)[0]["boundingbox"]
        bbox = [float(n) for n in bbox]
        return [bbox[2], bbox[0], bbox[3], bbox[1]]
    
    @staticmethod
    def seq_to_coordinates(seq):
        upper_left_long, upper_left_lat, bottom_right_long, bottom_right_lat = seq
        return upper_left_long, upper_left_lat, bottom_right_long, bottom_right_lat
    
    def bbox_to_x_range_y_range(self):
        x_range = [self.upper_left.long, self.bottom_right.long]
        y_range = [self.upper_left.lat, self.bottom_right.lat]
        return x_range, y_range
    
    def __repr__(self):
        return f'Upper left corner : {self.upper_left} , Bottom right corner : {self.bottom_right}'
    
class WGSBoundingBox(BoundingBox):
    """ WGS bounding box """
    world_bbox = [-178.3873749, -50.2187169, 172.3057152, 51.268318] 
    
    def __init__(self, upper_left, bottom_right):
        super().__init__(upper_left, bottom_right)
    
    @staticmethod
    def bbox_from_wgs_sequence(seq):
        ul_long, ul_lat, br_long, br_lat = BoundingBox.seq_to_coordinates(seq)
        upper_left = WGSCoordinates(ul_long, ul_lat)
        bottom_right = WGSCoordinates(br_long, br_lat)
        return WGSBoundingBox(upper_left, bottom_right)
    
    @staticmethod
    def bbox_from_mercator_sequence(seq):
        ul_long, ul_lat, br_long, br_lat = BoundingBox.seq_to_coordinates(seq)
        upper_left = WGSCoordinates.from_mercator_coordinates(ul_long, ul_lat)
        bottom_right = WGSCoordinates.from_mercator_coordinates(br_long, br_lat)
        return WGSBoundingBox(upper_left, bottom_right)
    
class MercatorBoundingBox(BoundingBox):
    """ Mercator bounding box """
    world_bbox = [-19857991.737816792, -6484240.26926857, 19180984.47683482, 6668894.01535062]
    
    def __init__(self, upper_left, bottom_right):
        super().__init__(upper_left, bottom_right)
    
    @staticmethod
    def bbox_from_wgs_sequence(seq):
        ul_long, ul_lat, br_long, br_lat = BoundingBox.seq_to_coordinates(seq)
        upper_left = MercatorCoordinates.from_wgs_coordinates(ul_long, ul_lat)
        bottom_right = MercatorCoordinates.from_wgs_coordinates(br_long, br_lat)
        return MercatorBoundingBox(upper_left, bottom_right)
        
    @staticmethod
    def bbox_from_mercator_sequence(seq):
        ul_long, ul_lat, br_long, br_lat = BoundingBox.seq_to_coordinates(seq)
        upper_left = MercatorCoordinates(ul_long, ul_lat)
        bottom_right = MercatorCoordinates(br_long, br_lat)
        return MercatorBoundingBox(upper_left, bottom_right)